# 2D Smoothing and Mapping (SAM)

## Overview

This document describes a 2D Smoothing and Mapping (SAM) demo inspired by **Sola-18** (refer to [the paper](https://arxiv.org/abs/1812.01537) for details). A 3D version of this example is available in `se3_sam.cpp`, and a simpler localization example using EKF can be found in `se2_localization.cpp`.

We consider a robot in 2D space surrounded by a small number of punctual landmarks or _beacons_. The robot receives control actions in the form of axial and angular velocities and can measure the location of the beacons relative to its own reference frame.

## Variables and Equations

The robot pose $X_i $ is in $SE(2) $, and the beacon positions $b_k $ are in $\mathbb{R}^2 $:

$
X_i = \begin{pmatrix} R_i & t_i \\ 0 & 1 \end{pmatrix}
$
where $R_i $ is the rotation matrix and $t_i $ the translation vector.

$
b_k = (b_{x_k}, b_{y_k})
$
represents the beacon coordinates in the world frame.

### Control Signal
The control signal $u $ is a twist in $se(2) $, comprising longitudinal velocity $v_x $ and angular velocity $w $:

$
u = (v_x \cdot \Delta t, 0, w \cdot \Delta t)
$

The control is corrupted by additive Gaussian noise $u_{\text{noise}} $, with covariance $Q $:

$
Q = \text{diag}(\sigma_v^2, \sigma_s^2, \sigma_{\text{yaw}}^2)
$

Upon receiving a control $u $, a new robot pose is created at:

$
X_j = X_i \cdot \exp(u)
$

### Landmark Measurements
Landmark measurements are of the range and bearing type but represented in Cartesian form for simplicity:

$
y = (y_x, y_y)
$

The noise $n $ in measurements is zero-mean Gaussian, with covariance $R $. The rigid motion action $y_{ik} = h(X_i, b_k) = X_i^{-1} b_k $.

## Factor Graph Representation

The world comprises 5 landmarks, observed from different poses in the factor graph. Below is a graphical representation of the SAM problem:



            ------- b1
    b3    /         |
    |   /       b4  |
    | /       /    \|
    X0 ---- X1 ---- X2
    | \   /   \   /
    |   b0      b2



Here:
- $X_i $ are SE(2) robot poses.
- $b_k $ are $\mathbb{R}^2 $ landmarks or beacons.
- The segments indicate measurement factors:
  - Motion measurements from $X_i $ to $X_j $.
  - Landmark measurements from $X_i $ to $b_k $.
  - An absolute pose measurement from $X_0 $ to the origin of coordinates.
